In [28]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import math
import statsmodels.api as sm
import scipy.stats
from matplotlib.ticker import MultipleLocator
import matplotlib.pyplot as plt
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import train_test_split


# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/ee-769-assignment1/sample_submission.csv
/kaggle/input/ee-769-assignment1/test.csv
/kaggle/input/ee-769-assignment1/train.csv


In [29]:
df = pd.read_csv("/kaggle/input/ee-769-assignment1/train.csv")

In [30]:
x1 = df.drop(['Attrition','ID','EmployeeCount','EmployeeNumber'],axis = 1)
y = df[['Attrition']]
x= pd.get_dummies(x1, drop_first=True)
x.head()

,Age,DailyRate,DistanceFromHome,Education,EnvironmentSatisfaction,HourlyRate,JobInvolvement,JobLevel,JobSatisfaction,MonthlyIncome,...,JobRole_Laboratory Technician,JobRole_Manager,JobRole_Manufacturing Director,JobRole_Research Director,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative,MaritalStatus_Married,MaritalStatus_Single,OverTime_Yes
0,41,1102,1,2,2,94,3,2,4,5993,...,0,0,0,0,0,1,0,0,1,1
1,49,279,8,1,3,61,2,2,2,5130,...,0,0,0,0,1,0,0,1,0,0
2,37,1373,2,2,4,92,2,1,3,2090,...,1,0,0,0,0,0,0,0,1,1
3,33,1392,3,4,4,56,3,1,3,2909,...,0,0,0,0,1,0,0,1,0,1
4,27,591,2,1,1,40,3,1,2,3468,...,1,0,0,0,0,0,0,1,0,0


In [31]:
regr2 = sm.OLS(y,x)
model = regr2.fit()
print(model.summary())

                                 OLS Regression Results                                
Dep. Variable:              Attrition   R-squared (uncentered):                   0.393
Model:                            OLS   Adj. R-squared (uncentered):              0.366
Method:                 Least Squares   F-statistic:                              14.51
Date:                Fri, 28 Aug 2020   Prob (F-statistic):                    5.05e-79
Time:                        07:40:16   Log-Likelihood:                         -282.68
No. Observations:                1028   AIC:                                      653.4
Df Residuals:                     984   BIC:                                      870.5
Df Model:                          44                                                  
Covariance Type:            nonrobust                                                  
                                        coef    std err          t      P>|t|      [0.025      0.975]
------------------

In [32]:
p=model.pvalues
max=p[1]
i=1
for i in range(len(x.columns)):
    if max<=p[i] and p[i]>=0.05:
        max = p[i]
        var=x.columns[i]
        if max >= 0.05:
            x=x.drop(var,1) 
    
    


In [33]:
x.head()

,Age,DistanceFromHome,Education,EnvironmentSatisfaction,HourlyRate,JobInvolvement,JobLevel,JobSatisfaction,MonthlyIncome,NumCompaniesWorked,...,JobRole_Laboratory Technician,JobRole_Manager,JobRole_Manufacturing Director,JobRole_Research Director,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative,MaritalStatus_Married,MaritalStatus_Single,OverTime_Yes
0,41,1,2,2,94,3,2,4,5993,8,...,0,0,0,0,0,1,0,0,1,1
1,49,8,1,3,61,2,2,2,5130,1,...,0,0,0,0,1,0,0,1,0,0
2,37,2,2,4,92,2,1,3,2090,6,...,1,0,0,0,0,0,0,0,1,1
3,33,3,4,4,56,3,1,3,2909,1,...,0,0,0,0,1,0,0,1,0,1
4,27,2,1,1,40,3,1,2,3468,9,...,1,0,0,0,0,0,0,1,0,0


In [34]:
#regr2 = sm.OLS(y,x)
#model = regr2.fit()
#print(model.summary())

In [35]:
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(x.values, i) for i in range(x.shape[1])]
vif["features"] = x.columns
vif_values=vif['VIF Factor']
vif_features=vif['features']
vif

,VIF Factor,features
0,34.102182,Age
1,2.326844,DistanceFromHome
2,9.333553,Education
3,7.468398,EnvironmentSatisfaction
4,11.543302,HourlyRate
5,16.348165,JobInvolvement
6,66.565459,JobLevel
7,7.303124,JobSatisfaction
8,57.208994,MonthlyIncome
9,2.754240,NumCompaniesWorked


In [36]:
def calculate_vif(x):
    thresh = 5.0
    output = pd.DataFrame()
    k = x.shape[1]
    vif = [variance_inflation_factor(x.values, j) for j in range(x.shape[1])]
    for i in range(1,k):
        print("Iteration no.")
        print(i)
        print(vif)
        a = np.argmax(vif)
        print("Max VIF is for variable no.:")
        print(a)
        if vif[a] <= thresh :
            break
        if i == 1 :          
            output = x.drop(x.columns[a], axis = 1)
            vif = [variance_inflation_factor(output.values, j) for j in range(output.shape[1])]
        elif i > 1 :
            output = output.drop(output.columns[a],axis = 1)
            vif = [variance_inflation_factor(output.values, j) for j in range(output.shape[1])]
    return(output)
x_new = calculate_vif(x)

Iteration no.
1
[34.10218205555263, 2.326843966084421, 9.333553416289465, 7.4683979808478425, 11.54330180605976, 16.348164644880764, 66.56545877926891, 7.303124406371204, 57.20899436596023, 2.7542398073074987, 44.688102940426, 155.7363060622331, 7.515765294512666, 3.589733023235297, 15.99031840764483, 5.6942308733683635, 16.420788532530477, 10.90913163774642, 6.9011674901373485, 2.560464674174169, 6.553902539084275, 3.011464001137403, 8.40291071467849, 61.86442871885709, 38.241271163311794, 2.398822324063658, 1.9801980488258608, 1.2213190469803152, 1.3323616580942017, 3.5407770968685344, 4.005997824032992, 4.795205343513126, 2.2824182422512127, 3.038548464180197, 4.5234679648693765, 16.43344967110564, 5.133132676763658, 3.3317395459622245, 4.621584941333124, 1.4781295443835698]
Max VIF is for variable no.:
11
Iteration no.
2
[34.05833403771793, 2.3252956324096306, 9.319536804150232, 7.430849088194978, 11.462862620533935, 16.189333058623834, 66.07761296162454, 7.283581306314085, 57.1242

Iteration no.
14
[2.224549960945077, 6.646985075512922, 6.643938944520588, 2.4750417256064914, 6.567500212338431, 3.288944777698536, 6.88077516536713, 5.202816581466189, 5.8174009900658765, 2.331137109059377, 5.275303502803552, 1.2645953201352997, 2.259598785841497, 1.8329131847637243, 1.1724767227751751, 1.2827205057625977, 1.2515301261539873, 2.7793333048653492, 2.1145164821336078, 1.997424931423129, 1.7046839301708943, 2.9782955849080652, 3.60434193941986, 1.72382115104692, 2.988821114695802, 4.098174408832787, 1.4566096793725827]
Max VIF is for variable no.:
6
Iteration no.
15
[2.2171316446711646, 6.581091063059475, 6.590632274417191, 2.149173322443807, 6.49994025276591, 3.288623600511206, 5.18307825441088, 5.714370467842029, 2.29407267567447, 5.041088823073616, 1.2602407918866667, 2.2595778966378255, 1.8315816032759298, 1.1724200752671488, 1.2827100763423207, 1.2471363341437114, 2.7270305595291884, 1.8490434559978826, 1.9952611744701176, 1.58057085796779, 2.9225774352149516, 3.591

In [37]:
x_new.head()

,DistanceFromHome,NumCompaniesWorked,StockOptionLevel,TrainingTimesLastYear,YearsSinceLastPromotion,YearsWithCurrManager,BusinessTravel_Travel_Frequently,EducationField_Life Sciences,EducationField_Marketing,EducationField_Other,...,JobRole_Laboratory Technician,JobRole_Manager,JobRole_Manufacturing Director,JobRole_Research Director,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative,MaritalStatus_Married,MaritalStatus_Single,OverTime_Yes
0,1,8,0,0,0,5,0,1,0,0,...,0,0,0,0,0,1,0,0,1,1
1,8,1,1,3,1,7,1,1,0,0,...,0,0,0,0,1,0,0,1,0,0
2,2,6,0,3,0,0,0,0,0,1,...,1,0,0,0,0,0,0,0,1,1
3,3,1,0,3,3,0,1,1,0,0,...,0,0,0,0,1,0,0,1,0,1
4,2,9,1,3,2,2,0,0,0,0,...,1,0,0,0,0,0,0,1,0,0


In [38]:
x_train,x_test,y_train,y_test = train_test_split(x_new,y,test_size=0.25,random_state=0)

In [39]:
from sklearn.linear_model import LogisticRegression
logistic_regression= LogisticRegression()
logistic_regression.fit(x_train,y_train)
y_pred=logistic_regression.predict(x_test)

/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [40]:
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
random_forest = RandomForestClassifier(n_estimators = 3000, criterion = 'entropy', random_state = 0)
random_forest.fit(x_train,y_train)
y_pred2=random_forest.predict(x_test)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


In [41]:
from sklearn import metrics
print('Accuracy: ',metrics.accuracy_score(y_test, y_pred2))


Accuracy:  0.8560311284046692


In [42]:

from sklearn.svm import SVC
svc = SVC(kernel = 'rbf')

svc.fit(x_train, y_train) 
y_pred4=svc.predict(x_test)

/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [43]:
from sklearn.tree import DecisionTreeClassifier 
clf_gini = DecisionTreeClassifier(criterion = "gini", random_state = 100,max_depth=10, min_samples_leaf=2) 
clf_gini.fit(x_train,y_train)

y_pred3=clf_gini.predict(x_test)

In [44]:
df_test = pd.read_csv('/kaggle/input/ee-769-assignment1/test.csv')
x_test = df_test.drop(['ID','EmployeeCount','EmployeeNumber',],axis = 1)
X= pd.get_dummies(x_test)

M=X.loc[:,list(x_new.columns)]
M


,DistanceFromHome,NumCompaniesWorked,StockOptionLevel,TrainingTimesLastYear,YearsSinceLastPromotion,YearsWithCurrManager,BusinessTravel_Travel_Frequently,EducationField_Life Sciences,EducationField_Marketing,EducationField_Other,...,JobRole_Laboratory Technician,JobRole_Manager,JobRole_Manufacturing Director,JobRole_Research Director,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative,MaritalStatus_Married,MaritalStatus_Single,OverTime_Yes
0,9,3,2,2,0,7,0,0,0,1,...,1,0,0,0,0,0,0,0,0,0
1,8,1,1,5,9,9,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
2,9,4,1,1,4,3,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
3,2,2,0,2,0,0,0,1,0,0,...,1,0,0,0,0,0,0,0,1,1
4,1,1,0,2,4,7,1,1,0,0,...,0,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
436,23,4,1,3,0,3,1,0,0,0,...,1,0,0,0,0,0,0,1,0,0
437,6,4,1,5,1,7,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
438,4,1,1,0,0,3,0,1,0,0,...,0,0,1,0,0,0,0,1,0,1
439,2,2,0,3,0,8,1,0,0,0,...,0,0,0,0,0,1,0,1,0,0


In [45]:
y_pred =random_forest.predict(M)


In [46]:
output=pd.DataFrame(data={"ID":df_test["ID"],"Attrition":y_pred})
output.to_csv('outputra.csv',index=False)

In [47]:
output=pd.DataFrame(data={"ID":df_test["ID"],"Attrition":y_pred})
output.to_csv('output.csv',index=False)